In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
def load_data():
    file_path = "F://Confirm Ticket3//railway_ticket_prediction_data.csv"
    df = pd.read_csv(file_path)
    
    # Convert date columns to datetime
    df["Date of Booking"] = pd.to_datetime(df["Date of Booking"])
    df["Date of Journey"] = pd.to_datetime(df["Date of Journey"])
    
    # Feature Engineering: Days before journey
    df["Days Before Journey"] = (df["Date of Journey"] - df["Date of Booking"]).dt.days
    
    # Drop unnecessary columns
    df = df.drop(columns=["PNR Number", "Date of Booking", "Date of Journey"])
    
    return df

# Split data
def split_data(df):
    X = df.drop(columns=["Final Status"])
    y = df["Final Status"]
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

# Save model
def save_model(model):
    joblib.dump(model, "models/prediction_of_ticket_model.pkl")

# Main execution
df = load_data()
X_train, X_test, y_train, y_test = split_data(df)
model = train_model(X_train, y_train)
save_model(model)

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.2f}")
print(report)


Model Accuracy: 0.91
              precision    recall  f1-score   support

           0       0.81      1.00      0.90        79
           1       1.00      0.85      0.92       121

    accuracy                           0.91       200
   macro avg       0.91      0.93      0.91       200
weighted avg       0.93      0.91      0.91       200



In [2]:
import os

# Create the 'models' directory if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")

print("✅ 'models' folder created successfully!")


✅ 'models' folder created successfully!


In [3]:
joblib.dump(model, "models/prediction_of_ticket_model.pkl")
print("✅ Model saved successfully!")


✅ Model saved successfully!


In [4]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
def load_data():
    file_path = "F://Confirm Ticket3//railway_ticket_prediction_data.csv"
    df = pd.read_csv(file_path)
    
    # Convert date columns to datetime
    df["Date of Booking"] = pd.to_datetime(df["Date of Booking"])
    df["Date of Journey"] = pd.to_datetime(df["Date of Journey"])
    
    # Feature Engineering: Days before journey
    df["Days Before Journey"] = (df["Date of Journey"] - df["Date of Booking"]).dt.days
    
    # Drop unnecessary columns
    df = df.drop(columns=["PNR Number", "Date of Booking", "Date of Journey"])
    
    return df

# Split data
def split_data(df):
    X = df.drop(columns=["Final Status"])
    y = df["Final Status"]
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

# Save model
def save_model(model):
    joblib.dump(model, "models/prediction_of_ticket_model.pkl")

# Main execution
df = load_data()
X_train, X_test, y_train, y_test = split_data(df)
model = train_model(X_train, y_train)
save_model(model)

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.2f}")
print(report)


Model Accuracy: 0.91
              precision    recall  f1-score   support

           0       0.81      1.00      0.90        79
           1       1.00      0.85      0.92       121

    accuracy                           0.91       200
   macro avg       0.91      0.93      0.91       200
weighted avg       0.93      0.91      0.91       200



In [5]:
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model_path = "F:/Confirm Ticket3/models/prediction_of_ticket_model.pkl"
model = joblib.load(model_path)

# App Title
st.title("🚆 Railway Ticket Confirmation Predictor")

# Introduction
st.markdown(
    """
    Welcome! This tool predicts the probability of your railway ticket getting **confirmed**.
    Enter the details below and click **'Predict'** to get your result.
    """
)

# Booking Status Input
booking_status = st.selectbox(
    "Booking Status",
    options=[0, 1, 2, 3],  
    format_func=lambda x: ["Waiting List", "RAC", "Confirmed", "Tatkal"][x],
    help="Current booking status of your ticket:\n"
         "- **Waiting List** (0): Your ticket is on the waiting list.\n"
         "- **RAC** (1): Reservation Against Cancellation.\n"
         "- **Confirmed** (2): Your ticket is already confirmed.\n"
         "- **Tatkal** (3): Emergency ticket booking."
)

# Quota Selection
quota = st.selectbox(
    "Quota",
    options=[1, 2, 3, 4],  
    format_func=lambda x: ["General", "Tatkal", "Ladies", "Senior Citizen"][x - 1],
    help="Ticket booking quota:\n"
         "- **General** (1): Regular booking.\n"
         "- **Tatkal** (2): Emergency quota.\n"
         "- **Ladies** (3): Reserved for female passengers.\n"
         "- **Senior Citizen** (4): Reserved for elderly passengers."
)

# Train Type Selection
train_type = st.selectbox(
    "Train Type",
    options=[1, 2, 3],  
    format_func=lambda x: ["Express", "Superfast", "Rajdhani"][x - 1],
    help="Select the type of train:\n"
         "- **Express** (1): Regular fast train.\n"
         "- **Superfast** (2): Faster than express trains.\n"
         "- **Rajdhani** (3): Premium high-speed train."
)

# Passenger Age Input
age = st.number_input(
    "Passenger Age",
    min_value=0, max_value=100, value=20,
    help="Enter the passenger's age. Age can impact the ticket confirmation probability."
)

# Gender Selection
gender = st.selectbox(
    "Gender",
    options=[1, 2],
    format_func=lambda x: ["Male", "Female"][x - 1],
    help="Select the gender of the passenger."
)

# Days Before Journey Input
days_before = st.number_input(
    "Days Before Journey",
    min_value=0, max_value=365, value=2,
    help="Enter the number of days before the journey when the ticket was booked. Earlier bookings have higher confirmation chances."
)

# Prediction Button
if st.button("Predict Confirmation Probability"):
    # Create input array
    input_data = np.array([[booking_status, quota, train_type, age, gender, days_before]])
    
    # Get prediction
    probability = model.predict_proba(input_data)[0][1] * 100  # Convert to percentage
    
    # Display Result
    st.subheader("🎯 Prediction Result")
    st.write(f"Your ticket confirmation probability is **{probability:.2f}%**.")
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model_path = "F:/Confirm Ticket3/models/prediction_of_ticket_model.pkl"
model = joblib.load(model_path)

# App Title
st.title("🚆 Railway Ticket Confirmation Predictor")

# Introduction
st.markdown(
    """
    Welcome! This tool predicts the probability of your railway ticket getting **confirmed**.
    Enter the details below and click **'Predict'** to get your result.
    """
)

# Booking Status Input
booking_status = st.selectbox(
    "Booking Status",
    options=[0, 1, 2, 3],  
    format_func=lambda x: ["Waiting List", "RAC", "Confirmed", "Tatkal"][x],
    help="Current booking status of your ticket:\n"
         "- **Waiting List** (0): Your ticket is on the waiting list.\n"
         "- **RAC** (1): Reservation Against Cancellation.\n"
         "- **Confirmed** (2): Your ticket is already confirmed.\n"
         "- **Tatkal** (3): Emergency ticket booking."
)

# Quota Selection
quota = st.selectbox(
    "Quota",
    options=[1, 2, 3, 4],  
    format_func=lambda x: ["General", "Tatkal", "Ladies", "Senior Citizen"][x - 1],
    help="Ticket booking quota:\n"
         "- **General** (1): Regular booking.\n"
         "- **Tatkal** (2): Emergency quota.\n"
         "- **Ladies** (3): Reserved for female passengers.\n"
         "- **Senior Citizen** (4): Reserved for elderly passengers."
)

# Train Type Selection
train_type = st.selectbox(
    "Train Type",
    options=[1, 2, 3],  
    format_func=lambda x: ["Express", "Superfast", "Rajdhani"][x - 1],
    help="Select the type of train:\n"
         "- **Express** (1): Regular fast train.\n"
         "- **Superfast** (2): Faster than express trains.\n"
         "- **Rajdhani** (3): Premium high-speed train."
)

# Passenger Age Input
age = st.number_input(
    "Passenger Age",
    min_value=0, max_value=100, value=20,
    help="Enter the passenger's age. Age can impact the ticket confirmation probability."
)

# Gender Selection
gender = st.selectbox(
    "Gender",
    options=[1, 2],
    format_func=lambda x: ["Male", "Female"][x - 1],
    help="Select the gender of the passenger."
)

# Days Before Journey Input
days_before = st.number_input(
    "Days Before Journey",
    min_value=0, max_value=365, value=2,
    help="Enter the number of days before the journey when the ticket was booked. Earlier bookings have higher confirmation chances."
)

# Prediction Button
if st.button("Predict Confirmation Probability"):
    # Create input array
    input_data = np.array([[booking_status, quota, train_type, age, gender, days_before]])
    
    # Get prediction
    probability = model.predict_proba(input_data)[0][1] * 100  # Convert to percentage
    
    # Display Result
    st.subheader("🎯 Prediction Result")
    st.write(f"Your ticket confirmation probability is **{probability:.2f}%**.")




2025-03-20 12:10:59.762 
  command:

    streamlit run C:\Users\shree\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [6]:
import os
print("Files in this directory:", os.listdir())


Files in this directory: [' railway_ticket_ml.py', '.anaconda', '.android', '.cache', '.conda', '.condarc', '.continuum', '.emulator_console_auth_token', '.gradle', '.idlerc', '.ipynb_checkpoints', '.ipython', '.jupyter', '.m2', '.matplotlib', '.ms-ad', '.streamlit', '.vscode', 'anaconda3', 'Anaconda3-2024.06-1-Windows-x86_64.exe', 'AndroidStudioProjects', 'app.py', 'AppData', 'Application Data', 'backup.ipynb', 'Confirm Ticket', 'consolerailway.ipynb', 'consoletrainconifrmation', 'Contacts', 'Cookies', 'datamites.ipynb', 'Documents', 'Downloads', 'Favorites', 'final.py', 'hadoop-2.10.2.tar.gz', 'hadoop-3.4.0-src (1).tar.gz', 'hadoop-3.4.0-src.tar.gz', 'index.html', 'Links', 'Local Settings', 'Measure_of_central_tendencies_and_data_variability.ipynb', 'ml-associate.ipynb', 'models', 'Music', 'My Documents', 'myticket.ipynb', 'NetHood', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{a2332f18-cdbf-11ec-8680-002248483d79}.TM.blf', 'NTUSER.DAT{a2332f18-cdbf-11ec-8680-00224

In [7]:
import os
print(os.getcwd())  # Get current working directory
print(os.listdir())  # List all files & folders in the current directory


C:\Users\shree
[' railway_ticket_ml.py', '.anaconda', '.android', '.cache', '.conda', '.condarc', '.continuum', '.emulator_console_auth_token', '.gradle', '.idlerc', '.ipynb_checkpoints', '.ipython', '.jupyter', '.m2', '.matplotlib', '.ms-ad', '.streamlit', '.vscode', 'anaconda3', 'Anaconda3-2024.06-1-Windows-x86_64.exe', 'AndroidStudioProjects', 'app.py', 'AppData', 'Application Data', 'backup.ipynb', 'Confirm Ticket', 'consolerailway.ipynb', 'consoletrainconifrmation', 'Contacts', 'Cookies', 'datamites.ipynb', 'Documents', 'Downloads', 'Favorites', 'final.py', 'hadoop-2.10.2.tar.gz', 'hadoop-3.4.0-src (1).tar.gz', 'hadoop-3.4.0-src.tar.gz', 'index.html', 'Links', 'Local Settings', 'Measure_of_central_tendencies_and_data_variability.ipynb', 'ml-associate.ipynb', 'models', 'Music', 'My Documents', 'myticket.ipynb', 'NetHood', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{a2332f18-cdbf-11ec-8680-002248483d79}.TM.blf', 'NTUSER.DAT{a2332f18-cdbf-11ec-8680-002248483d79}.T

In [ ]:
%%writefile "F:/Confirm Ticket3/app.py"
import streamlit as st
import pandas as pd
import joblib

# Load trained model
model = joblib.load("F:/Confirm Ticket3/models/prediction_of_ticket_model.pkl")

st.title("🚆 Railway Ticket Confirmation Predictor")

# User input fields
booking_status = st.selectbox("Booking Status", [0, 1, 2, 3])
quota = st.selectbox("Quota", [1, 2, 3, 4])
train_type = st.selectbox("Train Type", [0, 1, 2, 3])
age = st.number_input("Passenger Age", min_value=0, max_value=100, step=1)
gender = st.selectbox("Gender", [0, 1])
days_before_journey = st.number_input("Days Before Journey", min_value=0, max_value=365, step=1)

# Predict button
if st.button("Predict Confirmation Probability"):
    input_data = pd.DataFrame([[booking_status, quota, train_type, age, gender, days_before_journey]],
                              columns=["Booking Status", "Quota", "Train Type", "Passenger Age", "Gender", "Days Before Journey"])
    probability = model.predict_proba(input_data)[0, 1] * 100  # Probability of confirmation
    st.write(f"### 🎯 Ticket Confirmation Probability: {probability:.2f}%")


In [ ]:
import os
print(os.listdir("F:/Confirm Ticket3/models"))


In [ ]:
import os
import joblib

# Ensure the model path is correct
model_path = os.path.join("F:/Confirm Ticket3/models", "prediction_of_ticket_model.pkl")
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}")

model = joblib.load(model_path)


In [ ]:
import os
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load dataset
file_path = "F://Confirm Ticket3//data/railway_ticket_prediction_data.csv"
df = pd.read_csv(file_path)

# Preprocess data
df["Date of Booking"] = pd.to_datetime(df["Date of Booking"])
df["Date of Journey"] = pd.to_datetime(df["Date of Journey"])
df["Days Before Journey"] = (df["Date of Journey"] - df["Date of Booking"]).dt.days
df = df.drop(columns=["PNR Number", "Date of Booking", "Date of Journey"])

# Train the model
X = df.drop(columns=["Final Status"])
y = df["Final Status"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Ensure "models" folder exists
os.makedirs("F://Confirm Ticket3//models", exist_ok=True)

# Save the model
joblib.dump(model, "F://Confirm Ticket3//models//prediction_of_ticket_model.pkl")

print("✅ Model saved successfully!")


In [ ]:
import os
print(os.listdir("F://Confirm Ticket3//data"))


In [ ]:
file_path = r"F:\Confirm Ticket3\data\railway_ticket_prediction_data.csv"
df = pd.read_csv(file_path)


In [ ]:
import pandas as pd

# Correct file path
file_path = r"F://Confirm Ticket3//railway_ticket_prediction_data.csv"
df = pd.read_csv(file_path)

# Display first few rows to verify
print(df.head())


In [ ]:
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Preprocess data
df["Date of Booking"] = pd.to_datetime(df["Date of Booking"])
df["Date of Journey"] = pd.to_datetime(df["Date of Journey"])
df["Days Before Journey"] = (df["Date of Journey"] - df["Date of Booking"]).dt.days
df = df.drop(columns=["PNR Number", "Date of Booking", "Date of Journey"])

# Train the model
X = df.drop(columns=["Final Status"])
y = df["Final Status"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Ensure "models" folder exists
os.makedirs("F:/Confirm Ticket3/models", exist_ok=True)

# Save the model
joblib.dump(model, "F:/Confirm Ticket3/models/prediction_of_ticket_model.pkl")

print("✅ Model trained and saved successfully!")


In [22]:
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model_path = "F:/Confirm Ticket3/models/prediction_of_ticket_model.pkl"
model = joblib.load(model_path)

# App Title
st.title("🚆 Railway Ticket Confirmation Predictor")

# Introduction
st.markdown(
    """
    Welcome! This tool predicts the probability of your railway ticket getting **confirmed**.  
    Enter the details below and click **'Predict'** to see your result.
    """
)

# Booking Status Input
st.subheader("Booking Status")
st.write("Select the current status of your booking:")
booking_status = st.selectbox(
    "Choose Booking Status",
    options=[0, 1, 2, 3],  
    format_func=lambda x: ["Waiting List", "RAC", "Confirmed", "Tatkal"][x],
)
st.write(
    "- **Waiting List (0):** Your ticket is on the waiting list.\n"
    "- **RAC (1):** Reservation Against Cancellation.\n"
    "- **Confirmed (2):** Your ticket is already confirmed.\n"
    "- **Tatkal (3):** Emergency ticket booking."
)

# Quota Selection
st.subheader("Quota")
st.write("Choose the quota under which the ticket was booked:")
quota = st.selectbox(
    "Select Quota",
    options=[1, 2, 3, 4],  
    format_func=lambda x: ["General", "Tatkal", "Ladies", "Senior Citizen"][x - 1],
)
st.write(
    "- **General (1):** Regular booking.\n"
    "- **Tatkal (2):** Emergency quota.\n"
    "- **Ladies (3):** Reserved for female passengers.\n"
    "- **Senior Citizen (4):** Reserved for elderly passengers."
)

# Train Type Selection
st.subheader("Train Type")
st.write("Select the type of train you are traveling in:")
train_type = st.selectbox(
    "Choose Train Type",
    options=[1, 2, 3],  
    format_func=lambda x: ["Express", "Superfast", "Rajdhani"][x - 1],
)
st.write(
    "- **Express (1):** Regular fast train.\n"
    "- **Superfast (2):** Faster than express trains.\n"
    "- **Rajdhani (3):** Premium high-speed train."
)

# Passenger Age Input
st.subheader("Passenger Age")
age = st.number_input(
    "Enter Age",
    min_value=0, max_value=100, value=20
)
st.write("Enter the age of the passenger. Age can impact ticket confirmation probability.")

# Gender Selection
st.subheader("Gender")
gender = st.selectbox(
    "Select Gender",
    options=[1, 2],
    format_func=lambda x: ["Male", "Female"][x - 1],
)
st.write("Select the gender of the passenger.")

# Days Before Journey Input
st.subheader("Days Before Journey")
days_before = st.number_input(
    "Enter Days Before Journey",
    min_value=0, max_value=365, value=2
)
st.write("Enter the number of days before the journey when the ticket was booked. Earlier bookings have higher confirmation chances.")

# Prediction Button
if st.button("Predict Confirmation Probability"):
    # Create input array
    input_data = np.array([[booking_status, quota, train_type, age, gender, days_before]])
    
    # Get prediction
    probability = model.predict_proba(input_data)[0][1] * 100  # Convert to percentage
    
    # Display Result
    st.subheader("🎯 Prediction Result")
    st.write(f"Your ticket confirmation probability is **{probability:.2f}%**.")


In [27]:
%%writefile "F:/Confirm Ticket3/app.py"

import streamlit as st
import joblib
import numpy as np

# Load the trained model
model_path = "F:/Confirm Ticket3/models/prediction_of_ticket_model.pkl"
model = joblib.load(model_path)

# Streamlit UI
st.set_page_config(page_title="Railway Ticket Confirmation Predictor", layout="wide")

st.markdown("# 🚆 Railway Ticket Confirmation Predictor")

st.write("### Enter details below to check the probability of ticket confirmation.")

booking_status = st.selectbox("**Booking Status** (0: Waiting List, 1: RAC, 2: Confirmed)", [0, 1, 2])
quota = st.selectbox("**Quota** (1: General, 2: Tatkal, 3: Ladies, etc.)", [1, 2, 3, 4, 5])
train_type = st.selectbox("**Train Type** (1: Express, 2: Superfast, 3: Rajdhani, etc.)", [1, 2, 3, 4])
passenger_age = st.number_input("**Passenger Age** (Enter Age in Years)", min_value=1, max_value=100, value=25)
gender = st.selectbox("**Gender** (0: Female, 1: Male)", [0, 1])
days_before_journey = st.number_input("**Days Before Journey** (Days left before travel)", min_value=0, max_value=120, value=30)

# Prediction Button
if st.button("Predict Confirmation Probability"):
    input_data = np.array([[booking_status, quota, train_type, passenger_age, gender, days_before_journey]])
    prediction = model.predict_proba(input_data)[0][1]  # Probability of confirmation

    st.success(f"✅ Probability of ticket confirmation: **{prediction*100:.2f}%**")


Overwriting F:/Confirm Ticket3/app.py
